In [1]:
import os
import pandas as pd


In [2]:
path = '../../data/narrativeQA/html_tag_removed'

filenames = [f.replace('.content', '') for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
print(len(filenames))

785


In [3]:
qaps = pd.read_csv('./narrative_qa/qaps.csv')
id_url = pd.read_csv('./narrative_qa/id_url.csv')

In [4]:
qaps = qaps[~qaps.document_id.isin(filenames)]


In [5]:
id_url.iloc[:,3].max()


2026606

In [6]:
qaps.to_csv('qaps_html.csv', index=None)
id_url.to_csv('id_url_html.csv', index=None)

In [7]:
summaries = pd.read_csv('./narrative_qa/third_party/wikipedia/summaries.csv')


In [8]:
summaries.columns
summaries = summaries[~summaries.document_id.isin(filenames)]

In [9]:
summaries.to_csv('summaries_html.csv', index=None)


In [10]:
len(summaries['summary'].max())

4055

In [11]:
from datasets import Dataset, DatasetDict
import os
import json
import csv
import pandas as pd

qa_dataset = []
for index, row in qaps.iterrows():
    context = summaries[summaries['document_id'] == row['document_id']]['summary'].values[0]
    question = row['question']
    answer = [row['answer1'], row['answer2']]
    qa_dataset.append(
        {
            'context': context,
            'question': question,
            'answer': answer
        }
    )

KeyboardInterrupt: 

In [ ]:
qa_dataset[1]

In [ ]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

n_samples = len(qa_dataset)
train_end = int(train_ratio * n_samples)
val_end = train_end + int(val_ratio * n_samples)
train_data = Dataset.from_list(qa_dataset[:train_end])
val_data = Dataset.from_list(qa_dataset[train_end:val_end])
test_data = Dataset.from_list(qa_dataset[val_end:])


In [ ]:
data_dict = DatasetDict({
    'train': train_data,
    'validation': val_data,
    'test': test_data
})

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-base')

def tokenize_example(example):
    context, question, answers = example['context'], example['question'], example['answer']
    
    print('context: ', context)
    print('question: ', question)
    print('answers: ', answers)
    

    input_str = f'context: {context} </s>'

    answer = ' | '.join(answers)
    print(answer)
    
    
    target_str = f'question: {question} answer: {answer} </s>'
    inputs = tokenizer(input_str, padding='max_length', truncation=True, max_length=5000, return_tensors='pt')
    target = tokenizer(target_str, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    return {'input_ids': inputs['input_ids'][0], 'attention_mask':inputs['attention_mask'][0], 'target_mask':target['attention_mask'][0]}


In [ ]:

tokenized_dataset = data_dict.map(tokenize_example)